In [1]:
import pandas as pd
import numpy as np
import requests
from selenium import webdriver  
from selenium.common.exceptions import NoSuchElementException  
from selenium.webdriver.common.keys import Keys  
from bs4 import BeautifulSoup
import json
import time
import os
import pickle
import pprint
from pyvirtualdisplay import Display #use linuxbrew to install chromedriver if you want to do this on AWS instead of your local machine

First, we'll use the Portland Maps API to get all homes sold in Portland over a two year span. https://www.portlandmaps.com/development/

In [2]:
! conda list

# packages in environment at C:\Users\e06764\AppData\Local\Continuum\anaconda3:
#
# Name                    Version                   Build  Channel
_anaconda_depends         2019.03                  py37_0  
_ipyw_jlab_nb_ext_conf    0.1.0                    py37_0  
alabaster                 0.7.12                   py37_0  
anaconda                  custom                   py37_1  
anaconda-client           1.7.2                    py37_0  
anaconda-navigator        1.9.6                    py37_0  
anaconda-project          0.8.2                    py37_0  
asn1crypto                0.24.0                   py37_0  
astroid                   2.1.0                    py37_0  
astropy                   3.1              py37he774522_0  
atomicwrites              1.2.1                    py37_0  
attrs                     18.2.0           py37h28b3542_0  
babel                     2.6.0                    py37_0  
backcall                  0.1.0                    py37_0  
backports  

In [17]:
# https://github.com/psf/requests/issues/3829
# alternate not tried: https://stackoverflow.com/questions/48391750/disable-python-requests-ssl-validation-for-an-imported-module 
# probably better: https://stackoverflow.com/questions/39356413/how-to-add-a-custom-ca-root-certificate-to-the-ca-store-used-by-pip-in-windows/52961564#52961564 
# https://curl.haxx.se/docs/sslcerts.html
class WrappedSession(requests.Session):
    """A wrapper for requests.Session to override 'verify' property, ignoring REQUESTS_CA_BUNDLE environment variable.

    This is a workaround for https://github.com/kennethreitz/requests/issues/3829 (will be fixed in requests 3.0.0)
    """
    def merge_environment_settings(self, url, proxies, stream, verify, *args, **kwargs):
        if self.verify is False:
            verify = False

        return super(WrappedSession, self).merge_environment_settings(url, proxies, stream, verify, *args, **kwargs)
    
#from requests.packages.urllib3.exceptions import InsecureRequestWarning
#requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [15]:
WrappedSession(requests.get('https://www.google.com',verify=False))

TypeError: __init__() takes 1 positional argument but 2 were given

In [20]:
requests.get('https://www.google.com',verify='C:/Users/e06764/Documents/R/R-3.5.2/etc/curl-ca-bundle.crt')

SSLError: HTTPSConnectionPool(host='www.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')])")))

In [22]:
#get all houses sold since between 7/1/16 and 6/30/17 from Portland Maps
addresses = []


kwargs = {'api_key': '93851FDB9C653383A25A53061B0FD80C',
    'sqft_larger': '1',
    'sold_after': '07/01/2016',
    'sold_before': '06/30/2017',
    'page':'1',
    'format': 'json'
    }
    
u = 'http://www.portlandmaps.com/api/assessor'
request = requests.get(u, params=kwargs)
js = json.loads(request.text)

print(js['total'])
    
    
for i in range(len(js['results'])):
    if (js['results'][i]['address']):
        addresses.append(js['results'][i])
            
if int(js['total']) > 1000:
    pages = int(js['total'])//1000
    
    for i in range(2,pages+2):
        kwargs = {'api_key': '###',
        'sqft_larger': '1',
        'sold_after': '07/01/2016',
        'sold_before': '06/30/2017',
        'page': i,
        'format': 'json'
    }
    
        u = 'https://www.portlandmaps.com/api/assessor'
        request = requests.get(u, params=kwargs, verify=False)
        js = json.loads(request.text)
    
        for i in range(len(js['results'])):
            if (js['results'][i]['address']):
                addresses.append(js['results'][i])
    


SSLError: HTTPSConnectionPool(host='www.portlandmaps.com', port=443): Max retries exceeded with url: /api/assessor?api_key=93851FDB9C653383A25A53061B0FD80C&sqft_larger=1&sold_after=07%2F01%2F2016&sold_before=06%2F30%2F2017&page=1&format=json (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')])")))

In [4]:
addresses = pd.DataFrame(addresses)
addresses = addresses[['address', 'city', 'latitude', 'longitude', 'legal_description', 'market_value', 'neighborhood', 'owner', 'property_id', 'sale_date', 'sale_price', 'square_feet', 'year_built', 'zip_code']]
addresses = addresses[addresses['city'] == 'PORTLAND']
addresses = addresses[addresses['neighborhood'] != 'HAYDEN ISLAND NEIGHBORHOOD NETWORK'] #get rid of floating homes
addresses.reset_index(drop=True, inplace=True)
print(len(addresses))
addresses.head()

15899


,address,city,latitude,longitude,legal_description,market_value,neighborhood,owner,property_id,sale_date,sale_price,square_feet,year_built,zip_code
0,0102 SW PENDLETON ST,PORTLAND,45.48100,-122.67765,"SOUTHPORT, BLOCK 5, W 1/2 OF LOT 21&22",663380.0,SOUTH PORTLAND,"SWANSON,OCTAVIA",R273655,2017-03-20,759000,2050,1994.0,97239.0
1,0110 SW BANCROFT ST,PORTLAND,45.49213,-122.67662,"PORTLAND HMSTD, SUB OF LOT 1, BLOCK 5 TL 4500,...",777240.0,SOUTH PORTLAND,BLB LLC,R247596,2016-07-13,1350000,4914,1962.0,97239.0
2,01100 SW PALATINE HILL RD,PORTLAND,45.44406,-122.66709,"PALATINE HILL, LOT 45 TL 1000",936930.0,MULT CO RIVERDALE AREA,"VENTURA,KATHLEEN L & O'NEILL,JAMES E",R232835,2017-06-07,1250000,4212,1943.0,97219.0
3,01115 SW COMUS ST,PORTLAND,45.44701,-122.66653,"PALATINE HILL 3, BLOCK 106, LOT 15&16",491710.0,MULT CO RIVERDALE AREA,"CARLSON,JORDAN & CARLSON,ALISHA",R232939,2016-09-13,650000,1533,1968.0,97219.0
4,01125 SW PALATINE HILL RD,PORTLAND,45.44515,-122.66694,"PALATINE HILL 3, BLOCK 104, LOT 4&5 TL 601",1233040.0,MULT CO RIVERDALE AREA,"FLAHERTY,RYAN",R658483,2017-05-03,1775000,4004,2016.0,97219.0


Now, we'll get the Portland Maps API details on each of the homes

In [11]:
#get full Portlandmaps detail of each of the above properties

count = 1
for i in range(len(addresses)):
    kwargs = {'api_key': '###',
    'detail_type': 'assessor',
    'sections': '*',
    'detail_id': addresses['property_id'][i],
    'format': 'json'
    }

    u = 'https://www.portlandmaps.com/api/detail'
    request = requests.get(u, params=kwargs)
    if request.status_code == 429:
        break
    
    if count % 150 == 0:
        count = 1
        time.sleep(600)
    else:    
        count = count + 1
        time.sleep(2)
    
    if count % 10 == 0:
        addresses.to_pickle('home_results.pkl')
        
    print(i,request)
    js = json.loads(request.text)
    try:
        addresses.set_value(i, 'prop_tax_yr_of_sale', js['tax ' 'history'][1]['property_tax'])
        addresses.set_value(i, 'lot_size', js['general']['total_land_area_sqft'])
        addresses.set_value(i, 'type', js['improvements']['building_class'])
        addresses.set_value(i, 'plumbing', js['improvements']['plumbing'])
        addresses.set_value(i, 'description',js['improvements']['improvement_type'])
        addresses.set_value(i, 'main_sqft', js['improvements']['details'][0]['area_sq_ft'])
        addresses.set_value(i, 'segments', js['improvements']['number_of_segments'])
    except:
        continue

13000 <Response [200]>
13001 <Response [200]>
13002 <Response [200]>
13003 <Response [200]>
13004 <Response [200]>
13005 <Response [200]>
13006 <Response [200]>
13007 <Response [200]>
13008 <Response [200]>
13009 <Response [200]>
13010 <Response [200]>
13011 <Response [200]>
13012 <Response [200]>
13013 <Response [200]>
13014 <Response [200]>
13015 <Response [200]>
13016 <Response [200]>
13017 <Response [200]>
13018 <Response [200]>
13019 <Response [200]>
13020 <Response [200]>
13021 <Response [200]>
13022 <Response [200]>
13023 <Response [200]>
13024 <Response [200]>
13025 <Response [200]>
13026 <Response [200]>
13027 <Response [200]>
13028 <Response [200]>
13029 <Response [200]>
13030 <Response [200]>
13031 <Response [200]>
13032 <Response [200]>
13033 <Response [200]>
13034 <Response [200]>
13035 <Response [200]>
13036 <Response [200]>
13037 <Response [200]>
13038 <Response [200]>
13039 <Response [200]>
13040 <Response [200]>
13041 <Response [200]>
13042 <Response [200]>
13043 <Resp

13357 <Response [200]>
13358 <Response [200]>
13359 <Response [200]>
13360 <Response [200]>
13361 <Response [200]>
13362 <Response [200]>
13363 <Response [200]>
13364 <Response [200]>
13365 <Response [200]>
13366 <Response [200]>
13367 <Response [200]>
13368 <Response [200]>
13369 <Response [200]>
13370 <Response [200]>
13371 <Response [200]>
13372 <Response [200]>
13373 <Response [200]>
13374 <Response [200]>
13375 <Response [200]>
13376 <Response [200]>
13377 <Response [200]>
13378 <Response [200]>
13379 <Response [200]>
13380 <Response [200]>
13381 <Response [200]>
13382 <Response [200]>
13383 <Response [200]>
13384 <Response [200]>
13385 <Response [200]>
13386 <Response [200]>
13387 <Response [200]>
13388 <Response [200]>
13389 <Response [200]>
13390 <Response [200]>
13391 <Response [200]>
13392 <Response [200]>
13393 <Response [200]>
13394 <Response [200]>
13395 <Response [200]>
13396 <Response [200]>
13397 <Response [200]>
13398 <Response [200]>
13399 <Response [200]>
13400 <Resp

13714 <Response [200]>
13715 <Response [200]>
13716 <Response [200]>
13717 <Response [200]>
13718 <Response [200]>
13719 <Response [200]>
13720 <Response [200]>
13721 <Response [200]>
13722 <Response [200]>
13723 <Response [200]>
13724 <Response [200]>
13725 <Response [200]>
13726 <Response [200]>
13727 <Response [200]>
13728 <Response [200]>
13729 <Response [200]>
13730 <Response [200]>
13731 <Response [200]>
13732 <Response [200]>
13733 <Response [200]>
13734 <Response [200]>
13735 <Response [200]>
13736 <Response [200]>
13737 <Response [200]>
13738 <Response [200]>
13739 <Response [200]>
13740 <Response [200]>
13741 <Response [200]>
13742 <Response [200]>
13743 <Response [200]>
13744 <Response [200]>
13745 <Response [200]>
13746 <Response [200]>
13747 <Response [200]>
13748 <Response [200]>
13749 <Response [200]>
13750 <Response [200]>
13751 <Response [200]>
13752 <Response [200]>
13753 <Response [200]>
13754 <Response [200]>
13755 <Response [200]>
13756 <Response [200]>
13757 <Resp

14071 <Response [200]>
14072 <Response [200]>
14073 <Response [200]>
14074 <Response [200]>
14075 <Response [200]>
14076 <Response [200]>
14077 <Response [200]>
14078 <Response [200]>
14079 <Response [200]>
14080 <Response [200]>
14081 <Response [200]>
14082 <Response [200]>
14083 <Response [200]>
14084 <Response [200]>
14085 <Response [200]>
14086 <Response [200]>
14087 <Response [200]>
14088 <Response [200]>
14089 <Response [200]>
14090 <Response [200]>
14091 <Response [200]>
14092 <Response [200]>
14093 <Response [200]>
14094 <Response [200]>
14095 <Response [200]>
14096 <Response [200]>
14097 <Response [200]>
14098 <Response [200]>
14099 <Response [200]>
14100 <Response [200]>
14101 <Response [200]>
14102 <Response [200]>
14103 <Response [200]>
14104 <Response [200]>
14105 <Response [200]>
14106 <Response [200]>
14107 <Response [200]>
14108 <Response [200]>
14109 <Response [200]>
14110 <Response [200]>
14111 <Response [200]>
14112 <Response [200]>
14113 <Response [200]>
14114 <Resp

14428 <Response [200]>
14429 <Response [200]>
14430 <Response [200]>
14431 <Response [200]>
14432 <Response [200]>
14433 <Response [200]>
14434 <Response [200]>
14435 <Response [200]>
14436 <Response [200]>
14437 <Response [200]>
14438 <Response [200]>
14439 <Response [200]>
14440 <Response [200]>
14441 <Response [200]>
14442 <Response [200]>
14443 <Response [200]>
14444 <Response [200]>
14445 <Response [200]>
14446 <Response [200]>
14447 <Response [200]>
14448 <Response [200]>
14449 <Response [200]>
14450 <Response [200]>
14451 <Response [200]>
14452 <Response [200]>
14453 <Response [200]>
14454 <Response [200]>
14455 <Response [200]>
14456 <Response [200]>
14457 <Response [200]>
14458 <Response [200]>
14459 <Response [200]>
14460 <Response [200]>
14461 <Response [200]>
14462 <Response [200]>
14463 <Response [200]>
14464 <Response [200]>
14465 <Response [200]>
14466 <Response [200]>
14467 <Response [200]>
14468 <Response [200]>
14469 <Response [200]>
14470 <Response [200]>
14471 <Resp

14785 <Response [200]>
14786 <Response [200]>
14787 <Response [200]>
14788 <Response [200]>
14789 <Response [200]>
14790 <Response [200]>
14791 <Response [200]>
14792 <Response [200]>
14793 <Response [200]>
14794 <Response [200]>
14795 <Response [200]>
14796 <Response [200]>
14797 <Response [200]>
14798 <Response [200]>
14799 <Response [200]>
14800 <Response [200]>
14801 <Response [200]>
14802 <Response [200]>
14803 <Response [200]>
14804 <Response [200]>
14805 <Response [200]>
14806 <Response [200]>
14807 <Response [200]>
14808 <Response [200]>
14809 <Response [200]>
14810 <Response [200]>
14811 <Response [200]>
14812 <Response [200]>
14813 <Response [200]>
14814 <Response [200]>
14815 <Response [200]>
14816 <Response [200]>
14817 <Response [200]>
14818 <Response [200]>
14819 <Response [200]>
14820 <Response [200]>
14821 <Response [200]>
14822 <Response [200]>
14823 <Response [200]>
14824 <Response [200]>
14825 <Response [200]>
14826 <Response [200]>
14827 <Response [200]>
14828 <Resp

15142 <Response [200]>
15143 <Response [200]>
15144 <Response [200]>
15145 <Response [200]>
15146 <Response [200]>
15147 <Response [200]>
15148 <Response [200]>
15149 <Response [200]>
15150 <Response [200]>
15151 <Response [200]>
15152 <Response [200]>
15153 <Response [200]>
15154 <Response [200]>
15155 <Response [200]>
15156 <Response [200]>
15157 <Response [200]>
15158 <Response [200]>
15159 <Response [200]>
15160 <Response [200]>
15161 <Response [200]>
15162 <Response [200]>
15163 <Response [200]>
15164 <Response [200]>
15165 <Response [200]>
15166 <Response [200]>
15167 <Response [200]>
15168 <Response [200]>
15169 <Response [200]>
15170 <Response [200]>
15171 <Response [200]>
15172 <Response [200]>
15173 <Response [200]>
15174 <Response [200]>
15175 <Response [200]>
15176 <Response [200]>
15177 <Response [200]>
15178 <Response [200]>
15179 <Response [200]>
15180 <Response [200]>
15181 <Response [200]>
15182 <Response [200]>
15183 <Response [200]>
15184 <Response [200]>
15185 <Resp

15499 <Response [200]>
15500 <Response [200]>
15501 <Response [200]>
15502 <Response [200]>
15503 <Response [200]>
15504 <Response [200]>
15505 <Response [200]>
15506 <Response [200]>
15507 <Response [200]>
15508 <Response [200]>
15509 <Response [200]>
15510 <Response [200]>
15511 <Response [200]>
15512 <Response [200]>
15513 <Response [200]>
15514 <Response [200]>
15515 <Response [200]>
15516 <Response [200]>
15517 <Response [200]>
15518 <Response [200]>
15519 <Response [200]>
15520 <Response [200]>
15521 <Response [200]>
15522 <Response [200]>
15523 <Response [200]>
15524 <Response [200]>
15525 <Response [200]>
15526 <Response [200]>
15527 <Response [200]>
15528 <Response [200]>
15529 <Response [200]>
15530 <Response [200]>
15531 <Response [200]>
15532 <Response [200]>
15533 <Response [200]>
15534 <Response [200]>
15535 <Response [200]>
15536 <Response [200]>
15537 <Response [200]>
15538 <Response [200]>
15539 <Response [200]>
15540 <Response [200]>
15541 <Response [200]>
15542 <Resp

15856 <Response [200]>
15857 <Response [200]>
15858 <Response [200]>
15859 <Response [200]>
15860 <Response [200]>
15861 <Response [200]>
15862 <Response [200]>
15863 <Response [200]>
15864 <Response [200]>
15865 <Response [200]>
15866 <Response [200]>
15867 <Response [200]>
15868 <Response [200]>
15869 <Response [200]>
15870 <Response [200]>
15871 <Response [200]>
15872 <Response [200]>
15873 <Response [200]>
15874 <Response [200]>
15875 <Response [200]>
15876 <Response [200]>
15877 <Response [200]>
15878 <Response [200]>
15879 <Response [200]>


Now, we'll use the Zillow API to get more details on each of the homes. You need to make two API calls (ugh!) -  one to get the Zillow ID, and one to get the home details. 

In [ ]:
#zillow keys
#get a lot of keys because zillow has a 1000 call per day per key limit

keys1 = ['###', '###', '###']

keys2 = ['###', '###', '###']

In [ ]:
count = 0
for i in range(len(addresses)):
    
    if i & 1000 == 0: #API limits
        count = count + 1
        key = keys1[count]
        
    print(i, key)

    kwargs = {'zws-id': key,
        'address': addresses['address'][i],
        'citystatezip': str(addresses['city'][i]) + ' OR ' + str(addresses['zip_code'][i])
        }

    u = 'http://www.zillow.com/webservice/GetDeepSearchResults.htm'
    request = requests.get(u, params=kwargs)

    time.sleep(2)

    try:
        
        soup = BeautifulSoup(request.text,'html.parser')
  
        addresses.set_value(i, 'zillow_id',  soup.find_all('zpid')[0].text)
        addresses.set_value(i, 'zillow_finished_sqft', soup.find_all('finishedsqft')[0].text)
        addresses.set_value(i,'zillow_bedrooms', soup.find_all('bedrooms')[0].text)

    except:
        continue    

In [ ]:
#use this Zillow API to get school and realtor info

count = 0
for i in range(len(addresses)):
    
    if i & 1000 == 0: #API limits
        count = count + 1
        key = keys2[count]
        
    print(count, key)
    
    try:
        zillow_id = int(addresses['zillow_id'][i])
        count = count + 1
        
        kwargs = {'zws-id': key,
        'zpid': zillow_id
        }

        u = 'http://www.zillow.com/webservice/GetUpdatedPropertyDetails.htm'
        request = requests.get(u, params=kwargs)       
        time.sleep(2)       
        print(i,request, count)
        soup = BeautifulSoup(request.text,'html.parser')
        request = request.text
    except:
        continue


    try:
        addresses.set_value(i, 'numrooms',  soup.find_all('numrooms')[0].text)
    except:
        pass
    try:
        addresses.set_value(i, 'roof',  soup.find_all('roof')[0].text)
    except:
        pass
    try:
        addresses.set_value(i, 'exteriormaterial',  soup.find_all('exteriormaterial')[0].text)
    except:
        pass
    try:
        addresses.set_value(i, 'parkingtype',  soup.find_all('parkingtype')[0].text)
    except:
        pass
    try:
        addresses.set_value(i, 'coveredparkingspaces',  soup.find_all('coveredparkingspaces')[0].text)
    except:
        pass
    try:
        addresses.set_value(i, 'heatingsources',  soup.find_all('heatingsources')[0].text)
    except:
        pass
    try:
        addresses.set_value(i, 'heatingsystem',  soup.find_all('heatingsystem')[0].text)
    except:
        pass
    try:
        addresses.set_value(i, 'coolingsystem',  soup.find_all('coolingsystem')[0].text)
    except:
        pass
    try:
        addresses.set_value(i, 'appliances',  soup.find_all('appliances')[0].text)
    except:
        pass
    try:
        addresses.set_value(i, 'floorcovering',  soup.find_all('floorcovering')[0].text)
    except:
        pass
    try:
        addresses.set_value(i, 'architecture',  soup.find_all('architecture')[0].text)
    except:
        pass
    try:
        addresses.set_value(i, 'homedescription',  soup.find_all('homedescription')[0].text)
    except:
        pass
    try:
        addresses.set_value(i, 'elementaryschool',  soup.find_all('elementaryschool')[0].text)
    except:
        pass
    try:
        addresses.set_value(i, 'middleschool',  soup.find_all('middleschool')[0].text)
    except:
        pass
    try:
        addresses.set_value(i, 'highschool',  soup.find_all('highschool')[0].text)
    except:
        pass


In [ ]:
addresses.to_pickle('zillow.pkl')

Now, we'll use need to get Redfin URLS of the houses in order to later download photos.

In [ ]:
for i in range(len(addresses)):
    if i % 100 == 0: #reload your browser once in a while
        chromedriver = "/usr/local/bin/chromedriver"
        driver = webdriver.Chrome(chromedriver)
#        display = Display(visible=0, size=(1300, 1080)) #use these lines for AWS instead of local
#        display.start()
#        chromedriver = "/home/ubuntu/.linuxbrew/bin/chromedriver"
#        driver = webdriver.Chrome(chromedriver)
#        driver.set_window_size(1300, 1080)
    try:
        url = 'https://images.google.com/'
#        driver.get("https://images.google.com/") #use this for AWS instead of local
        driver.get(url)

        search = driver.find_element_by_xpath('//*[@id="lst-ib"]')

        address = addresses['address'][i] + ' Portland OR Redfin'

        search.send_keys(address)

        search.send_keys(Keys.RETURN)  # XXX: problem here.

        pic = driver.find_element_by_xpath("//*[@id='rg_s']/div[1]/a/img")

        pic.click()
        time.sleep(5)
    
        download_pic = driver.find_element_by_xpath("//*[@id='irc_cc']/div[2]/div[1]/div[2]/div[2]/a/img")
        time.sleep(2.5)
    
        src = download_pic.get_attribute('src')
        print(i, src)
    
        addresses.set_value(i, 'url', src)
    except:
        continue

    if (i-1)%100 == 0:
        driver.close()


In [ ]:
addresses.to_pickle('zillow.pkl')

Now, get the photos. You need to hide your user agent from Redfin because they don't let you scrape directly


In [ ]:
#you must hide user agent from redfin!
for i in range(0,len(addresses)):
    link = addresses['url'][i]
    print(link)
    filename = 'pics/'+str(i)+'.png'
    if 'https://ssl.cdn-redfin.com' in addresses['url'][i]:
        !curl {link} -s -H 'Pragma: no-cache' -H 'Accept-Encoding: gzip, deflate, br' -H 'Accept-Language: en-US,en;q=0.8' -H 'Upgrade-Insecure-Requests: 1' -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36' -H 'Content-Type:application/x-www-form-urlencoded' -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8' -H 'Cache-Control: no-cache'  -H 'Connection: keep-alive' --compressed > {filename}
    else:
        !curl {link} > {filename}
    time.sleep(2)

In [ ]:
#if any files got messed up, try again:

#for files that don't exist:
for i in range(0,len(addresses)):
    link = addresses['url'][i]
    print(link)
    filename = 'pics/'+str(i)+'.png'
    if not os.path.exists(filename):
        if 'https://ssl.cdn-redfin.com' in link:
            !curl {link} -s -H 'Pragma: no-cache' -H 'Accept-Encoding: gzip, deflate, br' -H 'Accept-Language: en-US,en;q=0.8' -H 'Upgrade-Insecure-Requests: 1' -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36' -H 'Content-Type:application/x-www-form-urlencoded' -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8' -H 'Cache-Control: no-cache'  -H 'Connection: keep-alive' --compressed > {filename}
        else:
            urllib.request.urlretrieve(link, filename)
    time.sleep(2)

#for files that didnt download correctly:
for i in range(0,len(addresses)):
    link = addresses['url'][i]
    print(link)
    filename = 'pics/'+str(i)+'.png'
    if os.path.getsize(filename) < 500 :
        if 'https://ssl.cdn-redfin.com' in link:
            !curl {link} -s -H 'Pragma: no-cache' -H 'Accept-Encoding: gzip, deflate, br' -H 'Accept-Language: en-US,en;q=0.8' -H 'Upgrade-Insecure-Requests: 1' -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36' -H 'Content-Type:application/x-www-form-urlencoded' -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8' -H 'Cache-Control: no-cache'  -H 'Connection: keep-alive' --compressed > {filename}
        else:
            urllib.request.urlretrieve(link, filename)
    time.sleep(2)


Are any files still messed up? If so, delete those entries.

In [ ]:
indices = []
for i in range(len(addresses)):
    filename = 'pics/'+str(i)+'.png'
    if os.path.getsize(filename) < 500 or if not os.path.exists(filename):
        indices.append(i)
addresses = addresses.drop(addresses.index[indices])


In [ ]:
addresses.to_pickle('year2/home_results.pkl')
addresses.to_csv('year2/home_results.csv')